Dataset:
  
  https://s3.amazonaws.com/h2o-public-test-data/smalldata/chicago/Chicago_Ohare_International_Airport.csv
  
  https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv

Use the Chicago Weather and Airlines dataset for the following chanllenge.

   1. Import chicago weather and airlines dataset.
   1. Transform weather data's Date column to three new columns "Year", "Month", "DayofMonth".
   1. Question: What is the range of the Year column in airlines dataset?
   1. Question: What is the range of the Year column in weather dataset?
   1. Filter airlines data to only flights outgoing or incoming to Chicago, ORD.
   1. Run a join of the two frames as you see fit, specify in the comments if you ran an outer left, outer right, or inner join.
   1. Run a GLM model on the resulting merged frame, you can regress on columns such as "IsDepDelayed", "IsArrDelayed", or "WeatherDelay". Pick predictors at your discretion.
   1. Provide some insight on the resulting model which can be a classification or regression model.